In [23]:
import nibabel as nib
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import os
import pandas as pd

In [ ]:
def convert_single_img(filename):
    # Load NIfTI file
    nii = nib.load(filename) #'~/Downloads/IXI-T1/IXI012-HH-1211-T1.nii.gz'
    data = nii.get_fdata()

    # Save middle slice as PNG
    slice_idx = data.shape[2] // 2
    # Load your slice
    slice_data = data[:, :, slice_idx]
    # Rotate 90 degrees counter-clockwise
    slice_data = np.rot90(slice_data)

    plt.imsave('slice.png', slice_data, cmap='gray')

In [ ]:
def convert_all_img():
    # List all NIfTI files in the folder
    folder_path = '~/Downloads/IXI-T1'
    nii_files = [f for f in os.listdir(os.path.expanduser(folder_path)) if f.endswith('.nii.gz')]
    print(len(nii_files))

    # Load all NIfTI images into a list
    images = []
    for file in nii_files:
        nii = nib.load(os.path.join(os.path.expanduser(folder_path), file))
        data = nii.get_fdata()
        slice_idx = data.shape[2] // 2

        slice_data = data[:, :, slice_idx] # Load your slice
        slice_data = np.rot90(slice_data) # Rotate 90 degrees counter-clockwise

        image_fname = '../data/'+file
        image_fname = image_fname.replace(".nii.gz", ".png")
        plt.imsave(image_fname, slice_data, cmap='gray') #saves images to data directory

581


In [32]:
def convert_excel_to_csv(filename):
    excel_file = filename
    df = pd.read_excel(excel_file)
    #print(df.head())
    df.to_csv('../IXI.csv', index=False) # Save as .csv

In [33]:
convert_excel_to_csv("../IXI.xls")

In [39]:
def map_id(filename):
    # 1. Load the CSV
    df = pd.read_csv(filename)
    print("# of subjects", len(df))

    # 2. Get list of all image files in the data directory
    image_dir = '../data/'
    # Ensure we only pick up the images we created
    image_files = [f for f in os.listdir(image_dir) if f.endswith('.png')]

    # 3. Create a list of dictionaries to build a mapping dataframe
    file_mapping = []
    for filename in image_files:
        # Example filename: "IXI050-Guys-0711-T1.png"
        # Split by '-' and take the first part: "IXI050"
        id_part = filename.split('-')[0] 
        # Remove "IXI" and convert to integer: 50
        try:
            subject_id = int(id_part.replace('IXI', ''))
            file_mapping.append({'IXI_ID': subject_id, 'file_name': filename})
        except ValueError:
            continue # Skip files that don't match the naming convention

    # 4. Convert the mapping to a DataFrame
    df_files = pd.DataFrame(file_mapping)
    print("# of images", len(df_files))

    # 5. Merge with the original demographics dataframe
    # or 'left' join if we want to keep all rows from the CSV
    df_final = pd.merge(df, df_files, on='IXI_ID', how='left')

    # 6. Save the new combined CSV
    df_final.to_csv('IXI_with_filenames.csv', index=False)

    return df_final

In [40]:
df = map_id("../IXI.csv")
df.head()

# of subjects 619
# of images 581


,IXI_ID,"SEX_ID (1=m, 2=f)",HEIGHT,WEIGHT,ETHNIC_ID,MARITAL_ID,OCCUPATION_ID,QUALIFICATION_ID,DOB,DATE_AVAILABLE,STUDY_DATE,AGE,file_name
0,1,1,170,80,2,3,5,2,1968-02-22,0,NaN,NaN,NaN
1,2,2,164,58,1,4,1,5,1970-01-30,1,2005-11-18,35.800137,IXI002-Guys-0828-T1.png
2,12,1,175,70,1,2,1,5,1966-08-20,1,2005-06-01,38.781656,IXI012-HH-1211-T1.png
3,13,1,182,70,1,2,1,5,1958-09-15,1,2005-06-01,46.710472,IXI013-HH-1212-T1.png
4,14,2,163,65,1,4,1,5,1971-03-15,1,2005-06-09,34.236824,IXI014-HH-1236-T1.png
